## CNN: Fine-Tunning com HDC

### Imports

In [ ]:
from modules import utils, globals
import torch
from modules import encoders
from binhd.classifiers import BinHD
from modules.cifake import Cifake
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.calibration import LabelEncoder
from modules.encoders import RecordEncoder
from sklearn.metrics import classification_report

### Carregando features e labels do modelo pré-treinado

In [2]:
features = np.load('features.npy', mmap_mode='r')
labels = np.load('labels.npy', mmap_mode='r')

### Instanciando a classe Cifake

In [3]:
cifake = Cifake()
# Verifica as amostras e classes
print(f"Número de samples: {len(cifake.samples)}")
print(f"Classes encontradas: {cifake.features}")

Número de samples: 5000
Classes encontradas:         feat_0    feat_1    feat_2    feat_3    feat_4    feat_5    feat_6  \
0     1.275530  0.511700  0.316328  0.512983  0.419070  2.798556  0.381312   
1     0.783416  0.740631  1.052051  0.829430  0.354067  1.125743  0.059947   
2     1.658858  0.690397  0.531623  0.387227  0.063034  0.624954  0.011755   
3     0.493961  1.512607  0.857241  0.345536  0.647262  1.664678  1.051863   
4     1.212295  0.884584  0.540168  0.812452  1.249039  0.794421  1.210231   
...        ...       ...       ...       ...       ...       ...       ...   
4995  0.850414  1.955287  1.282537  0.515612  1.812142  1.640769  0.844919   
4996  0.078288  0.561694  0.763691  0.943355  0.600272  1.874586  0.322577   
4997  1.739780  0.491048  2.848875  0.527449  1.496410  2.979135  0.714684   
4998  0.284801  1.177762  2.298618  0.818732  2.214477  2.153216  0.404426   
4999  1.731809  0.139465  0.019743  1.590314  0.921707  0.660200  1.757970   

        feat_7    

### Valores mínimo e máximo das features numéricas

In [4]:
min_val, max_val = cifake.get_min_max_values()
print(min_val, max_val)

0.0 9.64875602722168


### Definindo hiperparâmetros

In [5]:
dimension = 10000
num_levels = 100
low = min_val
high = max_val
oper = "bind"

### Definindo X e y

In [6]:
X = cifake.features
print(X.shape)
print(X.dtypes.unique())

y = cifake.labels
le = LabelEncoder()
y_encoded = torch.tensor(le.fit_transform(y))


(5000, 512)
[dtype('float32')]


### Carregando o modelo BinHD

In [7]:
model = BinHD(dimension, cifake.num_classes)
print(X.dtypes)

feat_0      float32
feat_1      float32
feat_2      float32
feat_3      float32
feat_4      float32
             ...   
feat_507    float32
feat_508    float32
feat_509    float32
feat_510    float32
feat_511    float32
Length: 512, dtype: object


### Record Encoder

O RecordEncoder é responsável por codificar entradas contínuas em representações hiperdimensionais binárias ou densas. Ele recebe como entrada a dimensionalidade de saída (out_features), que define o tamanho dos vetores hiperdimensionais gerados. Além disso, utiliza o parâmetro size, que representa o número de atributos de entrada, e levels, que determina quantos níveis serão usados para discretizar os dados. Os parâmetros low e high definem os limites inferior e superior dos valores de entrada.

In [8]:
record_encoder = RecordEncoder(
            out_features=dimension,
            size=X.shape[1], 
            levels=num_levels,
            low=low,
            high=high
        )


A função run_encoders processa os dados em lotes, onde os dados de entrada são convertidos em tensores, codificados pelo record_encoder, e armazenados em uma lista para serem concatenados no final, retornando todos os vetores codificados.

In [ ]:
def run_encoders(X, device, record_encoder):

    assert isinstance(X, pd.DataFrame), "X precisa ser um DataFrame"
    assert not X.isnull().values.any(), "X contém valores NaN"
    assert device in ["cuda", "cpu"], f"Dispositivo inválido: {device}"

    encoded_batches = []

    record_encoder.to(device)
    record_encoder.eval()

    num_samples = len(X)

    with torch.no_grad():
        for i in range(0, num_samples, globals.BATCH_SIZE):
            end = min(i + globals.BATCH_SIZE, num_samples)

            x_batch_np = X.iloc[i:end].values.astype(np.float32)

            try:
                x_batch_tensor = torch.tensor(x_batch_np).to(device)
            except Exception as e:
                print(f"Erro ao converter batch {i}-{end} para tensor: {e}")
                continue

            try:
                encoded = record_encoder(x_batch_tensor)
                encoded_batches.append(encoded.cpu())
            except Exception as e:
                print(f"Erro ao codificar batch {i}-{end}: {e}")
                continue

            print(f"\rProcessando amostras {i} até {end}", end='', flush=True)

    if encoded_batches != []:
        encoded_all = torch.cat(encoded_batches, dim=0)
    else:
        raise RuntimeError("Nenhum batch foi codificado com sucesso.")

    return encoded_all


### Separando em treino e teste e rodando o encoder

In [11]:
X_record_encoder = run_encoders(X, globals.DEVICE, record_encoder)
labels = torch.tensor(y).to(globals.DEVICE)

y_encoded = torch.tensor(y_encoded).to(globals.DEVICE)

X_train, X_test, y_train, y_test = train_test_split(X_record_encoder, labels, test_size=0.3, random_state = 0)

Processando amostras 4992 até 5000

/tmp/ipykernel_8193/2993563554.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_encoded = torch.tensor(y_encoded).to(globals.DEVICE)


### Treinando o modelo

In [28]:
with torch.no_grad():
    model.fit(X_train,y_train)
    predictions = model.predict(X_test.to(torch.int8))  

y_pred_np = np.array(predictions)
y_test_np = np.array(y_test) 


/tmp/ipykernel_8193/3941384730.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  y_pred_np = np.array(predictions)
/tmp/ipykernel_8193/3941384730.py:6: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  y_test_np = np.array(y_test)


### Resultados e Conclusões

In [ ]:
print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test)
))

              precision    recall  f1-score   support

           0       0.51      0.14      0.22       777
           1       0.48      0.86      0.62       723

    accuracy                           0.48      1500
   macro avg       0.49      0.50      0.42      1500
weighted avg       0.49      0.48      0.41      1500

